## Tezio Wallet

Welcome to Tezio Wallet, an Arduino-based hardware wallet for the Tezos blockchain. Tezio Wallet is compatible with the Arduino MKR WiFi 1010 and the Arduino Nano 33 IoT, both of which include a cryptographic coprocessor to securely store keys and perform certain crytpographic functions. 

###  Installation

Install the Arduino IDE. Download and move the `TezioWallet` folder to your Arduino libraries folder, which is usually `My Documents\Arduino\libraries` on Windows or `Documents\Arduino\libraries` on macOS. Open the Arduino IDE and install the following dependencies using `Tools > Manage Libraries...`.

- ArduinoECCX08
- Crypto
- micro-ecc

### Usage

#### Setup

Running Tezio Wallet on your Arduino device requires that the device first be configured, provisioned, and locked. This is done using the `Tezio_Wallet_Setup.ino` sketch. The sketch runs an interactive setup process using the Arduino IDE's Serial Monitor to share data with the user and get user inputs. The process begins by loading default configuration data onto the Arduino's cryptographic coprocessor. Once the configuration data is written to the device, the user has the option to lock the cofiguration zone. This must be done before the device can be used. Note that the default configuration data stored in the `configuration.h` file is set up to enable current functionality but also to allow for possible future functionality such as encrypted writes to certain slots of the cryptochip's data zone. After the device is configured, the sketch proceeds to derive HD wallet cryptographic keys from a user supplied mnemonic phrase specifice in the `secrets.h` file, or if a mnemonic phrase isn't provided the sketch proceeds to derive a new 24 word phrase using entropy provided by the cryptochip's true random number generator. Mnemonic and key derivation are carried out using specifications outlined in the BIP-0039, BIP-0032, BIP-0044, SLIP-0044, and SLIP-0010. Secret keys are derived for all three elliptic curves supported by the Tezos blockchain: Ed25519, Secp256k1, and NIST P256 (Secp256r1). The keys are written to the Arduino's cryptochip. A user supplied read/write key is also written to the device. The read/write key will allow the user to perform encrypted reads and writes to certain data slots of the device after it is locked. After keys are written, the user is given the option to lock the cryptochip's data zone. After the data zone is locked, clear writes of cryptographic secrets will no longer be supported. The device must be locked before use. 

#### API

After the device is setup, provisioned, and locked, upload the `Tezio_Wallet_API.ino` sketch. The sketch can be run in debug (interactive) mode using the Arduino IDE's serial monitor. However, setting the debug flag to false puts the device into listening mode. It can then be connected via USB to any host machine and recieve and send data via serial. The API sketch invokes the TezioWallet_API class to expose certain cryptographic tools to the host device. Importantly, private (secret) keys never leave the device. In fact, the cryptochip implements hardware support for cryptographic functions using the NIST P256 curve so the NIST P256 secret key never leaves the cryptochip's secure element. This hardware support also means that cryptographic functions involving the NIST P256 curve are much faster than those of the other supported curves. Below are details about the structure of data packets sent and received using the API. This if followed by example interactions with a Tezio Wallet using python. 

### Packet Structure

#### To Wallet

Packets of bytes sent to the hardware wallet have four parts, one prefix byte, two length byte (LSB first), one or more body bytes, and two checksum bytes.

`packet = prefix (1 byte) + length (2 byte) + body (1 or more bytes) + checksum (2 bytes)`

The body is composed of an operation code (opCode), parameters, and data. 

`body = opCode (1 byte) + param1 (1 byte) + param2 (1 byte) + param3 (2 bytes) + data (1 or more bytes)`

A call may not require all parameters but if data is part of the body then values for all parameters must also be included. Parameter 3 is represented in code as a 16-bit variable but is always sent as two bytes with the LSB first. Packets are constructed as follows. First the body is constructed. The length bytes are the length of the body plus 3 to count both the length byte itself and the checksum bytes. The crc16 checksum is then computed for the length and body bytes. The checksum is appended LSB first. The prefix, which serves as a listening byte for the hardware wallet, is always 0x03. 

#### From Wallet

Packets of bytes received from the hardware wallet are similar but do not include a prefix since the host does not need to listen but simply waits for a reply to be sent. The body of the reply depends on the operation being executed. 

### Example Tezio Wallet Interactions using Python

At the time of writing, the Tezio Wallet API implements the following three operations:
- op_get_pk: Query the wallet for a public key corresonding to one of the secret private keys stored on the cryptochip. The public key returned can be raw bytes, compressed, base58 encoded, or as a Tezos public key hash (address). 
- op_sign: Send a message to the wallet for signing. The message can be raw bytes or pre-hashed by the host maching. The signature returned can be raw bytes or base58 encoded.
- op_verify: Send a message and signature to the wallet for signature verification. The message can be raw bytes or pre-hashed and the signature can be raw bytes or base58 encoded. 

Each of these operations is demonstrated below.

In [1]:
# import some useful python tools
from Tezio import TezioWallet

### Get Public Key Operation (op_get_pk)

| Packet Vars | Value |
|-------------|-------| 
| opCode      | 0x11  |
| param1      | curve |
| param2      | mode  |
| param3      | -     |
| data        | -     |


| curve | ECC curve |
|-------|-----------|
| 0x01  | Ed25519   |
| 0x02  | Secp256k1 |
| 0x03  | NIST P256 |

| mode | Public Key Format           |
|------|-----------------------------|
| 0x01 | Raw (32 or 64 bytes)        |
| 0x02 | Compressed (32 or 33 bytes) |
| 0x03 | Base58 Checksum Encoded     |
| 0x04 | Hashed (Tezos Address)      |

In [2]:
# retrieve the public key for curve NIST P256 in compressed format
opCode = 0x11
param1 = 0x03 
param2 = 0x02

myWallet = TezioWallet()
myWallet.build_packet(opCode, param1, param2)
    
print('Packet to be sent...')
print(myWallet.packet.hex())
print()

if (not myWallet.query_wallet()):
    print('Wallet query failed')

print('Response received...')
print(myWallet.response.hex())

Packet to be sent...
030700110302300a

Response received...
03105a7d89a3f6c5b3691dd055944556c9858041f86da391b01c8389115b5209f6


In [3]:
# retrieve the public key hash for the Ed25519 curve 
opCode = 0x11
param1 = 0x01 
param2 = 0x03

myWallet = TezioWallet()
myWallet.build_packet(opCode, param1, param2)

print('Packet to be sent...')
print(myWallet.packet.hex())
print()

if (not myWallet.query_wallet()):
    print('Wallet query failed')

print('Decoded response received...')
print(myWallet.response.decode('utf-8'))

Packet to be sent...
0307001101033606

Decoded response received...
edpkv1EQnd7cMykpzKuo8xEnioizPd3nP6YYoyoRyRmVK5Vg7TfsCB


In [4]:
# do the same but use the get_pk method
myWallet = TezioWallet(1) # argument is curve to use (param 1)
public_key = myWallet.get_pk(3) # argument is mode (param 2)
print(public_key.decode('utf-8'))

edpkv1EQnd7cMykpzKuo8xEnioizPd3nP6YYoyoRyRmVK5Vg7TfsCB


### Sign Operation (op_sign)

*param3 is not used but a value must be included whenever data is sent as part of the packet

| Packet Vars | Value  |
|-------------|--------| 
| opCode      | 0x21   |
| param1      | curve  |
| param2      | mode   |
| param3      | 0x0000 |
| data        | message|


| curve | ECC curve |
|-------|-----------|
| 0x01  | Ed25519   |
| 0x02  | Secp256k1 |
| 0x03  | NIST P256 |

| mode | message hashed | signature format        |
|------|----------------|-------------------------|
| 0x00 | N/A            | Default Signature (Base58 Checksum Encoded Zeros) |
| 0x01 | yes            | Raw (64 bytes)          |
| 0x02 | yes            | Base58 Checksum Encoded |
| 0x03 | no             | Raw (64 bytes)          |
| 0x04 | no             | Base58 Checksum Encoded |

### Verify Operation (op_verify)

| Packet Vars | Value              |
|-------------|--------------------| 
| opCode      | 0x22               |
| param1      | curve              |
| param2      | mode               |
| param3      | message length     |
| data        | message + signature|

| curve | ECC curve |
|-------|-----------|
| 0x01  | Ed25519   |
| 0x02  | Secp256k1 |
| 0x03  | NIST P256 |

| mode | message hashed | signature format        |
|------|----------------|-------------------------|
| 0x01 | yes            | Raw (64 bytes)          |
| 0x02 | yes            | Base58 Checksum Encoded |
| 0x03 | no             | Raw (64 bytes)          |
| 0x04 | no             | Base58 Checksum Encoded |

In [5]:
# get default signature for a curve (base58 checksum encoded zeros)
opCode = 0x21
param1 = 0x03
param2 = 0x00

myWallet = TezioWallet()
myWallet.build_packet(opCode, param1, param2);
print('Packet to be sent...')
print(myWallet.packet.hex())
print()

if (not myWallet.query_wallet()):
    print('Wallet query failed')

print('Base58 encoded signature from the body of the reply...')
print(myWallet.response.decode('utf-8'))


Packet to be sent...
030700210300ff89

Base58 encoded signature from the body of the reply...
p2sigMJWuMaj1zAfVMzdZzFnoncCKE7faHzJ7coB6h3ziUiGeZoTZUNfYSQR5t2dJ6cFWCvUx8CZdLRCigAUtrt2JEfRzvbDnL


In [6]:
# sign an unhased message using the Secp256k1 curve and getting a base58 checksum endoced result
opCode = 0x21
param1 = 0x02
param2 = 0x04
param3 = 0x0000 # not used but needed in packet since data is included
# data = bytearray('This is my message. There are many like it but this is mine', 'utf-8') # 32 bytes so hashed or unhashed mode works
data = bytearray('abcdefghijklmnopqrstuvwxyz012345', 'utf-8')

myWallet = TezioWallet()
myWallet.build_packet(opCode, param1, param2, param3, data);
print('Packet to be sent...')
print(myWallet.packet.hex())
print()

if (not myWallet.query_wallet()):
    print('Wallet query failed')

print('Base58 encoded signature from the body of the reply...')
print(myWallet.response.decode('utf-8'))

Packet to be sent...
03290021020400006162636465666768696a6b6c6d6e6f707172737475767778797a303132333435b15f

Base58 encoded signature from the body of the reply...
spsig1d8YQUt541rH7dQhUbyWXe16Le35AAs285pjrST8SqgzEMU1grH53hJLnsouk53brYsL45L4pCDJ5CxvjbzLcjzuERZvPx


In [8]:
# do the same with the op_sign method
# do the same but use the get_pk method
myWallet = TezioWallet(2) # argument is curve to use (param 1)
message = 'abcdefghijklmnopqrstuvwxyz012345'
mode = 4 # param 2
signature = myWallet.sign(mode, message) 
print(signature.decode('utf-8'))

spsig1JVPitvNkuGNCmv7ebo3Vv9HiDmHKCMaf3Vb9BRbBffVMuSRL4wNqxc1JFFwr6Kyinyv5gG8SM45gGbbsMuvix6ppczEu7


In [9]:
# verify the signature
opCode = 0x22
# param1 and param2 are unchanged
param3 = len(data) 
sig = myWallet.response # signature from last query
data = list(data) + list(sig) # the data is not the message signed with the signature appended

print(param3)
print(len(sig))

myWallet = TezioWallet()
myWallet.build_packet(opCode, param1, param2, param3, data);
print('Packet to be sent...')
print(myWallet.packet.hex())
print()

if (not myWallet.query_wallet()):
    print('Wallet query failed')

print('Signature valid (0x01) or invalid (0x00)...')
print(myWallet.response.hex())

32
99
Packet to be sent...
038c0022020420006162636465666768696a6b6c6d6e6f707172737475767778797a3031323334357370736967314a56506974764e6b75474e436d763765626f335676394869446d484b434d616633566239425262426666564d7553524c34774e717863314a46467772364b79696e797635674738534d343567476262734d75766978367070637a45753780e2

Signature valid (0x01) or invalid (0x00)...
01


### Example RPCs Using Tezio Wallet

SEND SOME TEZ TO THE WALLET ADDRESS FIRST

In [1]:
from Tezio import TezioWallet, TezioRPC

In [5]:
# send RPC to get counter
nodeURL = 'https://rpc.ghostnet.teztnets.xyz'
myWallet = TezioWallet(3) # using Ed25519 keys
myRPC = TezioRPC(nodeURL, myWallet)
myRPC.account

'tz3MyarJihHrejsze59J2Seita7jYWDCJDPe'

In [6]:
myRPC.constants()

{'proof_of_work_nonce_size': 8,
 'nonce_length': 32,
 'max_anon_ops_per_block': 132,
 'max_operation_data_length': 32768,
 'max_proposals_per_delegate': 20,
 'max_micheline_node_count': 50000,
 'max_micheline_bytes_limit': 50000,
 'max_allowed_global_constants_depth': 10000,
 'cache_layout_size': 3,
 'michelson_maximum_type_size': 2001,
 'preserved_cycles': 3,
 'blocks_per_cycle': 4096,
 'blocks_per_commitment': 32,
 'blocks_per_stake_snapshot': 256,
 'cycles_per_voting_period': 5,
 'hard_gas_limit_per_operation': '1040000',
 'hard_gas_limit_per_block': '5200000',
 'proof_of_work_threshold': '70368744177663',
 'tokens_per_roll': '6000000000',
 'seed_nonce_revelation_tip': '125000',
 'origination_size': 257,
 'baking_reward_fixed_portion': '5000000',
 'baking_reward_bonus_per_slot': '2143',
 'endorsing_reward_per_slot': '1428',
 'cost_per_byte': '250',
 'hard_storage_limit_per_operation': '60000',
 'quorum_min': 2000,
 'quorum_max': 7000,
 'min_proposal_quorum': 500,
 'liquidity_baking_

In [7]:
delegate = 'tz2UVeJk8RMBiUYRPuQRFQmMcCS4XDTBB8XJ'
results = myRPC.delegation(delegate)

Simulating operation...
Results...
{'contents': [{'kind': 'delegation', 'source': 'tz3MyarJihHrejsze59J2Seita7jYWDCJDPe', 'fee': '10000', 'counter': '11506162', 'gas_limit': '10000', 'storage_limit': '0', 'delegate': 'tz2UVeJk8RMBiUYRPuQRFQmMcCS4XDTBB8XJ', 'metadata': {'balance_updates': [{'kind': 'contract', 'contract': 'tz3MyarJihHrejsze59J2Seita7jYWDCJDPe', 'change': '-10000', 'origin': 'block'}, {'kind': 'accumulator', 'category': 'block fees', 'change': '10000', 'origin': 'block'}], 'operation_result': {'status': 'applied', 'consumed_gas': '1000', 'consumed_milligas': '1000000'}}}]}
Consumed Gas, Fee Estimate, Storage Estimate...
1000
385
41250
Preapply operation...
[{'contents': [{'kind': 'delegation', 'source': 'tz3MyarJihHrejsze59J2Seita7jYWDCJDPe', 'fee': '385', 'counter': '11506162', 'gas_limit': '1100', 'storage_limit': '41250', 'delegate': 'tz2UVeJk8RMBiUYRPuQRFQmMcCS4XDTBB8XJ', 'metadata': {'balance_updates': [{'kind': 'contract', 'contract': 'tz3MyarJihHrejsze59J2Seita7jY

In [5]:
destination = 'tz2Dy8HhPZZqmxVK5hE8Unc66fiWvqY8wFXN'
results = myRPC.send_mutez(5000000, destination)

Simulating operation...
Results...
{'contents': [{'kind': 'transaction', 'source': 'tz3MyarJihHrejsze59J2Seita7jYWDCJDPe', 'fee': '10000', 'counter': '11506161', 'gas_limit': '10000', 'storage_limit': '10000', 'amount': '5000000', 'destination': 'tz2Dy8HhPZZqmxVK5hE8Unc66fiWvqY8wFXN', 'metadata': {'balance_updates': [{'kind': 'contract', 'contract': 'tz3MyarJihHrejsze59J2Seita7jYWDCJDPe', 'change': '-10000', 'origin': 'block'}, {'kind': 'accumulator', 'category': 'block fees', 'change': '10000', 'origin': 'block'}], 'operation_result': {'status': 'applied', 'balance_updates': [{'kind': 'contract', 'contract': 'tz3MyarJihHrejsze59J2Seita7jYWDCJDPe', 'change': '-5000000', 'origin': 'block'}, {'kind': 'contract', 'contract': 'tz2Dy8HhPZZqmxVK5hE8Unc66fiWvqY8wFXN', 'change': '5000000', 'origin': 'block'}], 'consumed_gas': '1451', 'consumed_milligas': '1450040'}}}]}
Consumed Gas, Fee Estimate, Storage Estimate...
1451
43169
42900
Preapply operation...
[{'contents': [{'kind': 'transaction', 

In [5]:
fee = 10000
storageLimit = 10000
results = myRPC.reveal(fee, storageLimit)

Simulating operation...
RPC failed...
[{'kind': 'branch', 'id': 'proto.013-PtJakart.contract.previously_revealed_key', 'contract': 'tz2Dy8HhPZZqmxVK5hE8Unc66fiWvqY8wFXN'}]
RPC call for simulation failed...


In [ ]:
from Tezio import TezioWallet, TezioRPC
import binascii
import hashlib as hl

In [ ]:
# retrieve the public key and pkh (account) for the Ed25519 curve 
myWallet = TezioWallet(1) # 1 - Ed25519 curve
opCode = 0x11
param1 = 0x01 
param2 = 0x04
myWallet.build_packet(opCode, param1, param2)
myWallet.query_wallet()
account = myWallet.response.decode('utf-8')
print(account)

param2 = 0x03
myWallet.build_packet(opCode, param1, param2)
myWallet.query_wallet()
public_key = myWallet.response.decode('utf-8')
print(public_key)

In [ ]:
# send RPC to get counter
nodeURL = 'https://rpc.ghostnet.teztnets.xyz'
myWallet = TezioWallet(1) # using Ed25519 keys
myRPC = TezioRPC(nodeURL, myWallet)

### Reveal Operation

FIRST

data = {
'branch': 
'contents':
[
    'kind': 'reveal'
    'source':
    'fee'
    'counter'
    'gas_limit'
    'storage_limit'
    'public_key'
]
}

SIMULATE

data = {
'branch': 
'contents':
[
    'kind': 'reveal'
    'source':
    'fee'
    'counter'
    'gas_limit'
    'storage_limit'
    'public_key'
]
'signature': 
}

OPERATION

data = {
'operation': {
'branch': 
'contents':
[
    'kind': 'reveal'
    'source':
    'fee'
    'counter'
    'gas_limit'
    'storage_limit'
    'public_key'
]
'signature': 
}
'chain_id':
}

In [ ]:



# retreive chain_id, protocol, , coutner, etc.
header = myRPC.header()
chain_id = header['chain_id']
protocol = header['protocol']
branch = header['hash']
counter = myRPC.counter()
# increment counter
counter = str(int(counter)+1)

# construct parts
contents = []
contents += [{'kind': 'reveal', 'source': account, 'fee': '10000', 'counter': counter, 'gas_limit': '10000', 'storage_limit': '10000', 'public_key': public_key}]

In [ ]:
# construct operation
operation = {'branch': branch, 'contents': contents}
print(operation)


In [ ]:
# forge and sign
forgedOperation = '03' + myRPC.remote_forge(operation) # add prefix
binaryForgedOperation = binascii.unhexlify(forgedOperation)

# hash
b2b = hl.blake2b(digest_size = 32)
b2b.update(binaryForgedOperation)
myHash = b2b.digest()

#get signature of message
myWallet = TezioWallet()
opCode = 0x21
param1 = 0x01 
param2 = 0x02
param3 = 0x0000
data = bytearray(myHash)
myWallet.build_packet(opCode, param1, param2, param3, data)
myWallet.query_wallet()
signature = myWallet.response.decode('utf-8')
print(signature)

In [ ]:
# do run_operation
json = {}
operation['signature'] = signature
json['operation'] = operation
json['chain_id'] = chain_id
print(json)

result = myRPC.run_operation(json)
result

In [ ]:
# update parameters and signature 
# contents[0]['gas_limit'] = result['contents'][0]['metadata']['operation_result']['consumed_gas']
# construct operation
operation = {'branch': branch, 'contents': contents}
print(operation)
# forge and sign
forgedOperation = '03' + myRPC.remote_forge(operation) # add prefix
binaryForgedOperation = binascii.unhexlify(forgedOperation)
# hash
b2b = hl.blake2b(digest_size = 32)
b2b.update(binaryForgedOperation)
myHash = b2b.digest()
#get signature of message
myWallet = TezioWallet()
opCode = 0x21
param1 = 0x01 
param2 = 0x02
param3 = 0x0000
data = bytearray(myHash)
myWallet.build_packet(opCode, param1, param2, param3, data)
myWallet.query_wallet()
signature = myWallet.response.decode('utf-8')
print(signature)

param2 = 0x01
myWallet.build_packet(opCode, param1, param2, param3, data)
myWallet.query_wallet()
binarySignature = myWallet.response
print(binarySignature.hex())

In [ ]:
json = [{}]
json[0]['protocol'] = protocol
json[0]['branch'] = branch
json[0]['contents'] = contents
json[0]['signature'] = signature
json

In [ ]:
result = myRPC.preapply(json)
result

In [ ]:
# inject 
# import requests

#URL = '{nodeURL}/injection/operation'.format(nodeURL = nodeURL)
params = {'chain': 'main'}
data = binascii.hexlify(binaryForgedOperation[1:] + binarySignature).decode('utf-8')
data = '"' + data + '"'
print(data)
# r = requests.Request('POST', URL, params = params, data = data)
# prep = r.prepare()
# print(prep.url)
# print(prep.body)

result = myRPC.injection_operation(params, data)
print(result)

In [ ]:
#inject


print(data)
result = myRPC.injection_operation(params, data)
print(result)


In [ ]:
result['contents'][0]['metadata']['operation_result']['consumed_gas']

In [ ]:
gas_limit = operation['contents'][0]['gas_limit']

In [ ]:
gas_limit

In [ ]:
str(int(int(gas_limit)*1.2))